# Running a Decision Tree on A Larger Dataset

In [3]:
from SEData.data import *
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from time import sleep
from random import randint

In [5]:
questions = pd.read_csv('most_recent_by_category.csv', squeeze = True, index_col=0)

In [14]:
data_links = back_generate_links(questions['url'],10000)


In [42]:
data = pd.DataFrame([get_text(link) for link in data_links])


1 loop, best of 3: 26.6 s per loop


In [43]:
data.dropna(axis = 0, how = 'any', inplace = True) #Drop NAs in place.
data.columns = ['labels','text']

In [44]:
train_data, test_data = train_test_split(data, test_size = .25) #stratify = data['labels'])

In [45]:
train_corpus = train_data['text']
train_labels = train_data['labels']
test_corpus = test_data['text']
test_labels = test_data['labels']

In [46]:
tfidf = TfidfVectorizer(stop_words = 'english', max_features = 500) #Limit number of features at 500 words with highest Tfidf score
train_matrix = tfidf.fit_transform(train_corpus)

In [47]:
test_matrix = tfidf.transform(test_corpus)


## Train a Decision Tree Model
### X_train : train_matrix
### y_train : train_labels
### X_test : test_matrix
### y_test : test_labels

In [48]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier()
forest.fit(train_matrix, train_labels)

pred_labels = forest.predict(test_matrix)

In [49]:
print(accuracy_score(test_labels, pred_labels))

0.0


In [50]:
pd.DataFrame({"50 Test Data Labels":test_labels[:50], "50 Predicted Data Labels":pred_labels[:50]})

,50 Predicted Data Labels,50 Test Data Labels
25,boardgames,puzzling
35,health,worldbuilding
16,boardgames,japanese
24,boardgames,politics
30,boardgames,tex
3,boardgames,chemistry
0,health,academia
31,boardgames,travel
11,boardgames,gaming
